# Ndmg Tutorial: Running Inside Python

This tutorial provides a basic overview of how to run ndmg manually within Python. <br>
We begin by checking for dependencies,
then we set our input parameters,
then we smiply run the pipeline.

Running the pipeline is quite simple: call `ndmg_dwi_pipeline.ndmg_dwi_worker` with the correct arguments. <br>


Note that, although you can run the pipeline in Python, the absolute easiest way (outside Gigantum) is to run the pipeline from the command line once all dependencies are installed using the following command: <br>
`ndmg_bids </absolute/input/dir> </absolute/output/dir>`. <br>
This will run a single session from the input directory, and output the results into your output directory.

But for now, let's look at running in Python -- <br>
Let's begin!

In [1]:
import os
import os.path as op
import glob
import shutil
import warnings
import subprocess
from pathlib import Path

from ndmg.scripts import ndmg_dwi_pipeline
from ndmg.utils import s3_utils

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Check for dependencies, Set Directories

The below code is a simple check that makes sure AFNI and FSL are installed. <br>
We also set the input, data, and atlas paths.

#### Make sure that AFNI and FSL are installed

In [2]:
# FSL
try:
    print(f"Your fsl directory is located here: {os.environ['FSLDIR']}")
except KeyError:
    raise AssertionError("You do not have FSL installed! See installation instructions here: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation")
    
# AFNI
try:
    print(f"Your AFNI directory is located here: {subprocess.check_output('which afni', shell=True, universal_newlines=True)}")
except subprocess.CalledProcessError:
    raise AssertionError("You do not have AFNI installed! See installation instructions here: https://afni.nimh.nih.gov/pub/dist/doc/htmldoc/background_install/main_toc.html")

Your fsl directory is located here: /usr/share/fsl/5.0
Your AFNI directory is located here: /opt/afni/afni



#### Set Input, Output, and Atlas Locations

In [3]:
# Make data directory if it doesn't exist
ndmg_dir = Path("/mnt/labbook/input/")
data_dir = ndmg_dir / "data/BNU1"
atlas_dir = Path("/ndmg_atlases")


# Set an output directory
outdir = '/mnt/labbook/input/overview_untracked'
print(f"Your output directory will be : {outdir}")

Your output directory will be : /mnt/labbook/input/overview_untracked


## Choose input parameters

#### Naming Conventions
Here, we define input variables to the pipeline.
To run the `ndmg` pipeline, you need four files:
1. a `t1w` - this is a high-resolution anatomical image.
2. a `dwi` - the diffusion image.
3. bvecs - this is a text file that defines the gradient vectors created by a DWI scan.
4. bvals - this is a text file that defines magnitudes for the gradient vectors created by a DWI scan.

The naming convention is in the [BIDs](https://bids.neuroimaging.io/) spec.

In [4]:
# Specify base directory and paths to input files (dwi, bvecs, bvals, and t1w required)
subject_id = 'sub-0025864'

# Define the location of our input files.
t1w = str(data_dir / f"{subject_id}/ses-1/anat/{subject_id}_ses-1_T1w.nii.gz")
dwi = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.nii.gz")
bvecs = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.bvec")
bvals = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.bval")

print(t1w)
print(dwi)
print(bvecs)
print(bvals)

/mnt/labbook/input/data/BNU1/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz
/mnt/labbook/input/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.nii.gz
/mnt/labbook/input/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bvec
/mnt/labbook/input/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bval


### Parameter Choices and Output Directory
Here, we choose the parameters to run the pipeline with.
If you are inexperienced with diffusion MRI theory, feel free to just use the default parameters.

- *atlases = ['desikan', 'CPAC200', 'DKT', 'HarvardOxfordcort', 'HarvardOxfordsub', 'JHU', 'Schaefer2018-200', 'Talairach', 'aal', 'brodmann', 'glasser', 'yeo-7-liberal', 'yeo-17-liberal']* : The atlas that defines the node location of the graph you create.
- *mod_types = ['det', 'prob']* : Deterministic or probablistic tractography.
- *track_types = ['local', 'particle']* : Local or particle tracking.
- *mods = ['csa', 'csd']* : [Constant Solid Angle](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4360965/) or [Constrained Spherical Deconvolution](https://onlinelibrary.wiley.com/doi/10.1002/ima.22005).
- regs = *['native', 'native_dsn', 'mni']* : Registration style. If native, do all registration in each scan's space; if mni, register scans to the MNI atlas; if native_dsn, do registration in native space, and then fit the streamlines to MNI space.
- vox_size = *['1mm', '2mm']* : Whether our voxels are 1mm or 2mm.
- seeds = int : Seeding density for tractography. More seeds generally results in a better graph, but at a much higher computational cost.

In [5]:
# Use the default parameters.
atlas = 'desikan'
mod_type = 'prob'
track_type = 'local'
mod_func = 'csd'
reg_style = 'native'
vox_size = '2mm'
seeds = 1


#### Get masks and labels
The pipeline needs these two variables as input. <br>
Running the pipeline via `ndmg_bids` does this for you.

In [6]:
# Auto-set paths to neuroparc files
mask = str(atlas_dir / "atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz")
labels = [str(i) for i in (atlas_dir / "atlases/label/Human/").glob(f"*{atlas}*2x2x2.nii.gz")]

print(f"mask location : {mask}")
print(f"atlas location : {labels}")

mask location : /ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz
atlas location : ['/ndmg_atlases/atlases/label/Human/desikan_space-MNI152NLin6_res-2x2x2.nii.gz']


## Run the pipeline!

In [ ]:
ndmg_dwi_pipeline.ndmg_dwi_worker(dwi=dwi, bvals=bvals, bvecs=bvecs, t1w=t1w, atlas=atlas, mask=mask, labels=labels, outdir=outdir, vox_size=vox_size, mod_type=mod_type, track_type=track_type, mod_func=mod_func, seeds=seeds, reg_style=reg_style, clean=False, skipeddy=True, skipreg=True)

dwi = /mnt/labbook/input/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.nii.gz
bvals = /mnt/labbook/input/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bval
bvecs = /mnt/labbook/input/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bvec
t1w = /mnt/labbook/input/data/BNU1/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz
atlas = desikan
mask = /ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz
labels = ['/ndmg_atlases/atlases/label/Human/desikan_space-MNI152NLin6_res-2x2x2.nii.gz']
outdir = /mnt/labbook/input/overview_untracked
vox_size = 2mm
mod_type = prob
track_type = local
mod_func = csd
seeds = 1
reg_style = native
clean = False
skip eddy = True
skip registration = True
Output directory: /mnt/labbook/input/overview_untracked
Adding directory tree...
Connectomes downsampled to given labels: /mnt/labbook/input/overview_untracked/dwi/roi-connectomes/desikan_space-MNI152NLin6_res-2x2x2/sub-0025864_ses-1_dwi_desikan_space-MNI152NLin6_res-2

# Try It Yourself : Command Line

`ndmg` runs best as a standalone program on the command line.

The simplest form of the command, given that you have input data, pass an output folder, and have all dependencies installed, is the following:
    
    ndmg_bids </absolute/input/dir> </absolute/output/dir>
    
Here, we'll show you how to set this up yourself.

## Setup: Running Locally

1. install [FSL](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation)
2. install [AFNI](https://afni.nimh.nih.gov/pub/dist/doc/htmldoc/background_install/install_instructs/index.html)
3. `git clone https://github.com/neurodata/ndmg.git`
4. `cd ndmg`
5. `pip install -r requirements.txt`
6. `pip install .`

## Running Locally

### Most Basic

This will run the first session from your input dataset, and put the results into the output dataset.
We still recommend the `--atlas` flag so that graphs don't get generated on all possible atlases.

    ndmg_bids --atlas desikan </absolute/input/dir> </absolute/output/dir>

### Specifying Participant and Session

You can also specify a particular participant and session.
(This is extremely useful for setting up batch scripts to run large datasets).

    ndmg_bids --atlas desikan --participant_label <label> --session_label <number> </absolute/input/dir> </absolute/output/dir>

### Different Registration Styles, Diffusion Models, Tractography Styles

You can use:
- the `--sp` flag to set the registration space;
- the `--mf` flag to set the diffusion model; and
- the `--mod` flag to set deterministic / probablistic tracking;


    ndmg_bids --atlas desikan --sp <space> --mf <model> --mod <tracking style> </absolute/input/dir> </absolute/output/dir>

## Setup: Running in Docker

If you're having problems installing the program locally, it's often easier to use Docker.

1. install [docker](https://docs.docker.com/install/)
2. `docker pull neurodata/ndmg_dev:latest`

## Running in Docker

### Option A (Docker executable approach):
Once you've downloaded the docker image, you can:
Attach your local input and output folders with `-v`, <br>
Run the image, <br>
and input your participant and session labels into the container.

    docker run -ti --rm --privileged -e DISPLAY=$DISPLAY -v <absolute/path/to/input/data>:/input -v <absolute/path/to/output/data>:/outputs neurodata/ndmg_dev:latest --participant_label <label> --session_label <number> --atlas desikan /input /output

### Option B (Inside Docker container):

You can also enter the container yourself and then run `ndmg` from inside the container.

    docker run -ti --rm --privileged --entrypoint /bin/bash -e DISPLAY=$DISPLAY -v <absolute/path/to/input/data>/input -v <absolute/path/to/output/data>/output ndmg_dev:latest

    ndmg_bids --participant_label <label> --session_label <number> --atlas desikan /input /output